#### Introduction to Statistical Learning, Lab 4.2

# Logistic Regression


We will fit a logistic regression on the `Smarket` data set, trying to predict `Direction` using `Lag1` through `Lag5` and `Volume`. We do this in several different ways to learn about the data preparation we need to do for libraries other than `statsmodels`.



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from islpy import datasets, utils, lmplots
sns.set()
%matplotlib inline

We first load the data set.

In [ ]:
smarket = datasets.Smarket()
smarket.head()

The `smf.glm()` function constructs a *generalised linear model*.

In [ ]:
logit_fit = smf.glm('Direction~Lag1+Lag2+Lag3+Lag4+Lag5+Volume',
                    smarket, family=sm.families.Binomial()).fit()
logit_fit.summary()

Note that `statsmodels` (actually `patsy`) has automatically created an encoding for the `Up` and `Down` categories in `Direction`.

The smalles $p$-values is associated with `Lag1`.  The negative coefficient suggest that if the market had a positive response yesterday it is less likely to go up today. However, the $p$-values is relatively large and there is no clear evidence of a real association between `Lag1` and `Direction`.

The `islpy.lmplots` module also provides R-style control plots for GLM's. The functions are the same; they detect automatically whether we supply a GLM model or a simple linear regression model. The plotted quantities are slightly different for GLM's.

In [ ]:
lmplots.plot(logit_fit)
plt.show()

Soon (in the very next lab) we will have to prepare *design matrices* in order to use libraries other than `statsmodels`. We can still use the formula mini language to do this, which is most useful if we want include interaction terms or non-linear transformations of the predictors. This approach also provides automatic encoding of categories in qualitative variables.

To do this we have to use `patsy` directly. We will do this now, replicating the model we used above and then do a logistic regression using the obtained design matrices (still using `statsmodels` in this case). 

Here is how we produce design matrices using `patsy`. The formula language is the same (it was `patsy` working behind the scenes all along). While not strictly necessary, it is often beneficial to create the matrices as `pandas` data frames.

In [ ]:
import patsy

In [ ]:
Y, X = patsy.dmatrices('Direction~Lag1+Lag2+Lag3+Lag4+Lag5+Volume', 
                       data=smarket, return_type='dataframe')

In [ ]:
X.head()

In [ ]:
Y.head()

We see that `patsy` added an intercept to $X$ and encoded the categories in $Y$. We also note that the encoding is redundant in this case, so we modify $Y$ to only contain the column for `Up` before we fit the model. (After all we just want to predict the probabilities for `Up`).

In [ ]:
Y.drop(Y.columns[0], axis=1, inplace=True)
Y.head()

Now we are ready to fit the model.

In [ ]:
lm = sm.GLM(Y, X, family=sm.families.Binomial()).fit()
lm.summary()

Note that the signs of the coefficients are flipped compared to our previous model. This is due to the different category encoding. The control plots will therefore also look different.

In [ ]:
lmplots.plot(lm)
plt.show()

We can access the coefficients like this.

In [ ]:
lm.params

The `predict()` method is used to predict the *probabilities* of the market going up for each observation (if you need the raw logit predictions you can use `linear=True`). If no new data is provided, the data set used to fit the model is used for the prediction. 

In [ ]:
pred = lm.predict()
pred

We can easily convert this to a Boolean, telling us when the market is more likely to go up than down. 

In [ ]:
up = pred > 0.5
up

From this we can produce a `pandas.Series` with the string values `'Up'` and `'Down'`, if required.

In [ ]:
pred_str = pd.Series(up).replace({True: 'Up', False: 'Down'})
pred_str.head()

In order to determine how many observations were correctly or incorrectly classified, we produce a *confusion matrix*.

We will dip into the `sklearn.metrics` module for this. We will use the `sklearn` library more often from now on. 

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(up, lm.model.endog)

It might be useful and more readable to produce a `pandas` data frame.

In [ ]:
labels = ['Down', 'Up']
cm = pd.DataFrame(confusion_matrix(up, lm.model.endog), columns=labels, index=labels)
cm

We can compute the fraction of *correct* predictions by adding the diagonal elements of the confusion matrix and dividing by the number of observations.

In [ ]:
(cm.iloc[0, 0] + cm.iloc[1, 1]) / lm.nobs

We also compute this as the mean of the observations where the prediction matches the truth.

In [ ]:
(up == lm.model.endog).mean()

The fraction of *wrong* predictions is computed from the off-diagonal elements of the confusion matrix. (This is obviously the same a one minus the correct prediction rate).

In [ ]:
(cm.iloc[0, 1] + cm.iloc[1, 0]) / lm.nobs

In [ ]:
(cm.iloc[0, 0] + cm.iloc[1, 1]) / lm.nobs

Since we made the predictions on the training data set this number corresponds to the *training error rate*. This raises another question. The 52% correct prediction rate seems to suggest that our predictions are slightly better than random guesses. But this is misleading (otherwise we'd all be rich)! 

To properly assess how well our model does, we need to *split* the data set into a *training* and a *test* data set. We chose to use the data from before 2005 for training and the 2005 data for testing (after all we want to predict the future from the past). 


In [ ]:
Y_train = Y[smarket.Year < 2005]
Y_test = Y[smarket.Year == 2005]
X_train = X[smarket.Year < 2005]
X_test = X[smarket.Year == 2005]

In [ ]:
lm = sm.GLM(Y_train, X_train, family=sm.families.Binomial()).fit()
lm.summary()

Not surprisingly, the parameters are now slightly different, and so are the control plots.

In [ ]:
lmplots.plot(lm)
plt.show()

We now make our prediction on the *independent* test data set.

In [ ]:
up = lm.predict(X_test) > 0.5
cm = pd.DataFrame(confusion_matrix(up, Y_test), columns=labels, index=labels)
cm

The *test error rate* is then computed as follows.

In [ ]:
(cm.iloc[0, 1] + cm.iloc[1, 0]) / len(Y_test)

This means the model performs worse than random guessing on the test data!

The predictive power of our model is somewhat underwhelming. So we'll try a simpler model including only the two most influential predictors, namely `Lag1` and `Lag2`.

In [ ]:
Y, X = patsy.dmatrices('Direction~Lag1+Lag2', 
                       data=smarket, return_type='dataframe')
Y.drop(Y.columns[0], axis=1, inplace=True)
Y_train = Y[smarket.Year < 2005]
Y_test = Y[smarket.Year == 2005]
X_train = X[smarket.Year < 2005]
X_test = X[smarket.Year == 2005]
lm = sm.GLM(Y_train, X_train, family=sm.families.Binomial()).fit()
lm.summary()

In [ ]:
lmplots.plot(lm)
plt.show()

To asses the performance of the model we again produce the confusion matrix from predictions on the test data set.

In [ ]:
up = lm.predict(X_test) > 0.5
cm = pd.DataFrame(confusion_matrix(up, Y_test), columns=labels, index=labels)
print(cm)
print(f'correct: {100*(cm.iloc[0, 0] + cm.iloc[1, 1]) / len(Y_test):.2f}%')
print(f'error: {100*(cm.iloc[0, 1] + cm.iloc[1, 0]) / len(Y_test):.2f}%')

Given the 56% correct predictions it seems that the `Direction~Lag1+Lag2` model does very well. It is worth noting, though, that a naive cut-based model performs equally well. That is, our fancy fit does not do anything useful.

Another interesting observation is that in the cases the model predicts the market to go up it is correct 58% of the time. This seems to suggest that we can base a trading strategy on these cases. We do not recommend to bet your money on it without further investigation -- this might be a random fluctuation. 

In [ ]:
correct_up = 100 * cm.iloc[1, 1] / (cm.iloc[1, 0] + cm.iloc[1, 1]) 
correct_up

Sometimes we want to run some simple tests that involve quickly getting the prediction for a single observation or a small number of observations.

Let's say we want to get the prediction for two days where `Lag1` has the values $(1.2, 1.5)$ and `Lag2` has the values $(1.1, -0.8)$. If we have set up our model like we did, this is unfortunately a bit convoluted. The only sure fire way to do this is to create `patsy` design matrix from a `pandas` data frame with the proper formula (ie. the right hand side of the formula we used to construct model design matrices). 

In [ ]:
x = pd.DataFrame({'Lag1': [1.2, 1.5], 'Lag2': [1.1, -0.8]})
x = patsy.dmatrix('Lag1+Lag2', x, return_type='dataframe')
lm.predict(x)